# HydroBucket 模型测试

这个 notebook 展示了 HydroBucket 水文模型的基本使用方法，包括：
- 单节点模型的设置和运行
- 多节点模型的设置和运行
- 参数梯度计算示例

In [3]:
# 导入必要的包
using Pkg
Pkg.activate(".")
Pkg.instantiate()

using CSV
using DataFrames
using ComponentArrays
using ModelingToolkit
using BenchmarkTools
using DataInterpolations
using Zygote
using ForwardDiff

# 导入 HydroModels
include("../src/HydroModels.jl")

# 导入常用类型
HydroFlux = HydroModels.HydroFlux
StateFlux = HydroModels.StateFlux
HydroBucket = HydroModels.HydroBucket
HydroModel = HydroModels.HydroModel

# 导入模型定义
include("../models/exphydro.jl")

  Activating project at `e:\JlCode\HydroModels\notebook`
    Updating registry at `D:\Julia\packages\registries\General.toml`
   Installed StatsFuns ───────────── v1.4.0
   Installed ProgressMeter ───────── v1.10.4
   Installed OrdinaryDiffEqCore ──── v1.22.0
   Installed NaNMath ─────────────── v1.1.3
   Installed Optimisers ──────────── v0.4.6
   Installed SparseMatrixColorings ─ v0.4.16
   Installed NLSolversBase ───────── v7.9.1
   Installed NNlib ───────────────── v0.9.30
   Installed SciMLBase ───────────── v2.82.0
   Installed Symbolics ───────────── v6.37.1
   Installed ColorTypes ──────────── v0.12.1
   Installed Graphs ──────────────── v1.12.1
   Installed SymbolicUtils ───────── v3.25.1
   Installed StructArrays ────────── v0.7.1
   Installed Latexify ────────────── v0.16.7
   Installed ReverseDiff ─────────── v1.16.1
   Installed Optim ───────────────── v1.12.0
   Installed Lux ─────────────────── v1.12.1
    Updating `E:\JlCode\HydroModels\notebook\Project.toml`
  [6e4b80f

## 模型参数设置

下面我们设置模型的基本参数：
- f: 渗透系数
- Smax: 最大存储容量
- Qmax: 最大流量
- Df: 扩散系数
- Tmax: 最高温度阈值
- Tmin: 最低温度阈值

In [10]:
# 设置基本参数
ele = bucket_1
f, Smax, Qmax, Df, Tmax, Tmin = 0.01674478, 1709.461015, 18.46996175, 2.674548848, 0.175739196, -2.092959084

# 创建参数向量
params = ComponentVector(f=f, Smax=Smax, Qmax=Qmax, Df=Df, Tmax=Tmax, Tmin=Tmin)[HydroModels.get_param_names(ele)]
init_states = ComponentVector(snowpack=100.0)[HydroModels.get_state_names(ele)]
pas = ComponentVector(params=params, initstates=init_states)

# 读取输入数据
file_path = "../data/exphydro/01013500.csv"
data = CSV.File(file_path)
df = DataFrame(data)
ts = collect(1:10000)

# 准备单节点输入数据
input = (lday=df[ts, "dayl(day)"], temp=df[ts, "tmean(C)"], prcp=df[ts, "prcp(mm/day)"])
input_arr = reduce(hcat, collect(input[HydroModels.get_input_names(ele)])) |> permutedims


3×10000 Matrix{Float64}:
 3.1     4.24       8.02   15.27   8.48   …   11.55     1.84        0.14
 6.08   10.53      11.835   7.38   4.8       -11.475  -10.585     -17.79
 0.472   0.470151   0.468   0.464  0.464       0.416    0.419205    0.42

In [ ]:
# 创建单节点模型
single_ele = HydroBucket(name=:surface, fluxes=fluxes_1, dfluxes=dfluxes_1)

# 运行模型
result = single_ele(input_arr, pas)

  3.407 ms (148095 allocations: 7.37 MiB)


5×10000 Matrix{Float64}:
 0.0      0.0      0.0       0.0      …  277.874     279.714     279.854
 0.0      0.0      0.0       0.0          11.55        1.84        0.14
 3.1      4.24     8.02     15.27          0.0         0.0         0.0
 0.0      0.0      0.0       0.0          -0.0        -0.0        -0.0
 1.13779  1.51019  1.63204   1.21771       0.288362    0.311022    0.176836